In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import train_test_split
from  sklearn.ensemble import RandomForestClassifier

Считываем граф

In [2]:
vertex_num = 11566
train = pd.read_csv('./social_network.csv')

In [3]:
G = csr_matrix((np.ones(len(train)), (np.array(train['i']), np.array(train['j']))), 
                   shape=(vertex_num + 1, vertex_num + 1))

for i in range(vertex_num):
    if G[i, i]:
        G[i, i] = 0
G.eliminate_zeros()

Индекс Адамика-Адара

In [6]:
def predict_adamic_adar(graph, X_train, y_train, X_test, eps=0.1):
    adj_vertives = [set(graph[i].nonzero()[1]) for i in range(vertex_num + 1)]
    y_pred = []
    for (i, j) in X_test:
        coeff = 0
        for w in (adj_vertives[i] & adj_vertives[j]):
            coeff += (1 / (np.log2(len(adj_vertives[w])) + eps))
        y_pred.append(coeff)
        
    return np.array(y_pred)

Бейзлайн на основе числа путей длины 2

In [7]:
def predict_two_path(graph, X_train, y_train, X_test):
    G = graph.dot(graph)
    y_pred = []
    for (i, j) in X_test:
        y_pred.append(G[i, j])
        
    return np.array(y_pred)

Random Forest на простых признаках

In [8]:
def get_features(graph, edges):
    adj_vertives = [set(graph[i].nonzero()[1]) for i in range(vertex_num + 1)]
    
    features = []
    
    for (i,  j) in edges:
        cur_features = []
        
        cur_features.append(len(adj_vertives[i]))
        cur_features.append(len(adj_vertives[j]))
        cur_features.append(len(adj_vertives[i]) * len(adj_vertives[j]))
        cur_features.append(len(adj_vertives[j] & adj_vertives[i]))
        cur_features.append(len(adj_vertives[j] & adj_vertives[i]) / (len(adj_vertives[j] | adj_vertives[i]) + 2))
        
        features.append(cur_features)
    
    features = np.array(features)
    return features

def predict_rf(graph, X_train, y_train, X_test):
    clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1)
    clf.fit(get_features(graph, X_train), y_train)
    
    return clf.predict_proba(get_features(graph, X_test))[:, 1]

Кросс-валидация: рассматриваем все ребра $(i, j)$, такие что между вершинами $i$ и $j$ есть путь длины 2, из них выбираем  по $ 10^4 $ существующих и несуществующих ребер в выборку.

In [9]:
def get_edges(graph):
    two_path = graph.dot(graph)
    
    edges = [(i, j) for i, j in zip(*graph.nonzero())]
    edges_set = set(edges)
    
    possible_edges = [(i, j) for i, j in zip(*two_path.nonzero()) if (i, j) not in edges_set]
    return edges, possible_edges

def cross_validate(predict_func, n_samples=10 ** 4, n_iter=10):
    edges, possible_edges = get_edges(G)
    total_score = 0
    
    for _ in range(n_iter):
        positive_indice = np.random.choice(np.arange(len(edges)), n_samples, replace=False)
        negative_incide = np.random.choice(np.arange(len(possible_edges)), n_samples, replace=False)

        graph = G.copy()
        for i in positive_indice:
            x, y = edges[i]
            graph[x, y] = 0
        graph.eliminate_zeros()

        X = [edges[i] for i in positive_indice] + [possible_edges[i] for i in negative_incide]
        y = np.concatenate((np.ones(n_samples), np.zeros(n_samples)))

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

        y_pred = predict_func(graph, X_train, y_train, X_test)

        total_score += roc_auc_score(y_test, y_pred)
    return total_score / n_iter

Результаты кросс-валидации:

In [10]:
cross_validate(predict_adamic_adar) # 0.897 на public

0.54047679898879242

In [11]:
cross_validate(predict_two_path) # 0.721 на public

0.39116556659701629

In [12]:
cross_validate(predict_rf) # 0.72 на public

0.62073259743934317

In [18]:
test = pd.read_csv('./suspicious_edges.csv')
result = pd.read_csv('./ans.csv')

test_edges = [(i, j) for i, j in zip(test['i'], test['j'])]

Считаем ответы для тестовой выборки с помощью RandomForest. Обучающей выборки для начального графа у нас нет, поэтому сгенерируем несколько обучающих выборок как для кросс-валидации, а потом усредним результаты.

In [22]:
n_samples = 10 ** 4
n_iter = 10

res = np.zeros(len(test_edges))

for _ in range(n_iter):
    positive_indice = np.random.choice(np.arange(len(edges)), n_samples, replace=False)
    negative_incide = np.random.choice(np.arange(len(possible_edges)), n_samples, replace=False)

    graph = G.copy()
    for i in positive_indice:
        x, y = edges[i]
        graph[x, y] = 0
    graph.eliminate_zeros()

    X = [edges[i] for i in positive_indice] + [possible_edges[i] for i in negative_incide]
    y = np.concatenate((np.ones(n_samples), np.zeros(n_samples)))

    res += predict_rf(graph, X, y, test_edges)

res /= n_iter

In [31]:
result['prob'] = res
result.to_csv('sub21.csv', index=False)